In [1]:
import selenium
import numpy as np
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException, NoSuchElementException,StaleElementReferenceException
import time
import re

In [30]:
# 1
def views():
    driver=webdriver.Chrome()
    driver.get('https://en.wikipedia.org/wiki/List_of_most-viewed_YouTube_videos')
    try:
        Rank=[]
        Name=[]
        Artist=[]
        Upload_date=[]
        Views=[]
        table=WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//table[@class="sortable wikitable sticky-header static-row-numbers sort-under col3center col4right jquery-tablesorter"]/tbody')))
#         rows = table.find_elements(By.TAG_NAME, 'tr')
        for row in table:
            rows=row.find_elements(By.TAG_NAME,'tr')
        
            for data in rows:
                datas = data.find_elements(By.TAG_NAME, 'td')
                info = [cell.text for cell in datas]
                if info:
                    name=info[0].split('[')
                    Rank.append(name[1][:-1])
                    Name.append(name[0])
                    Artist.append(info[1])
                    Views.append(f"{info[2]} billion")
                    Upload_date.append(info[3])
                    
        df=pd.DataFrame({'Rank':Rank,'Name':Name,'Artist':Artist,'Views in (billions)':Views,'Upload Date':Upload_date})
        print(df)
    except Exception as e:
        print('An error occur',e)
        
views()

   Rank                                         Name  \
0     7                           "Baby Shark Dance"   
1    10                                  "Despacito"   
2    18                       "Johny Johny Yes Papa"   
3    19                                  "Bath Song"   
4    20                              "See You Again"   
5    25                               "Shape of You"   
6    28                          "Wheels on the Bus"   
7    29                "Phonics Song with Two Words"   
8    30                                "Uptown Funk"   
9    31                              "Gangnam Style"   
10   36  "Learning Colors – Colorful Eggs on a Farm"   
11   37                             "Dame Tu Cosita"   
12   38   "Masha and the Bear – Recipe for Disaster"   
13   39                                     "Axel F"   
14   40                        "Baa Baa Black Sheep"   
15   41                                      "Sugar"   
16   42                             "Lakdi Ki Ka

In [41]:
#2
def bcci():
    try:
        driver=webdriver.Chrome()
        driver.get('https://www.bcci.tv')
        fixture= WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/header/div[3]/div[2]/ul/div[1]/a[2]"))
        )
        fixture.click()
        time.sleep(10)
        
        Series=[]
        Place=[]
        Date=[]
        Time=[]
        morebtn=WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/section/div/div/div/div/div/div[2]/div[3]/div[2]/div/button"))
        )
        morebtn.click()
        time.sleep(10)
        info=driver.find_elements(By.XPATH,'//div[@class="match-info"]')
        try:
            for i in info:
                date=i.find_element(By.XPATH,'./div[1]/div[1]')
                Date.append(date.text)
                
                times=i.find_element(By.XPATH,'./div[1]/div[2]')
                Time.append(times.text)
                
                series=i.find_element(By.TAG_NAME,'h5')
                Series.append(series.text)
                
                place=i.find_element(By.XPATH,'./div[3]')
                Place.append(place.text)
                
            df=pd.DataFrame({'Date':Date,'Time':Time,'Series':Series,'Place':Place})
            print(df)
            
        except TimeoutException as t:
            print('time error',t)
        except NoSuchElementException as f:
            print('empty list',f)
        except StaleElementReferenceException as s:
            print('stale error',s)
            
    except Exception as e:
        print('An error occur',e)
        
bcci()

      Date       Time                                 Series  \
0    9 Jun  20:00 IST            ICC Mens T20 World Cup 2024   
1   12 Jun  20:00 IST            ICC Mens T20 World Cup 2024   
2   13 Jun  13:30 IST  South Africa Women Tour Of India 2024   
3   15 Jun  20:00 IST            ICC Mens T20 World Cup 2024   
4   16 Jun  13:30 IST  South Africa Women Tour Of India 2024   
5   19 Jun  13:30 IST  South Africa Women Tour Of India 2024   
6   23 Jun  13:30 IST  South Africa Women Tour Of India 2024   
7   28 Jun   9:30 IST  South Africa Women Tour Of India 2024   
8    5 Jul  19:00 IST  South Africa Women Tour Of India 2024   
9    6 Jul  20:00 IST            India Tour Of Zimbabwe 2024   
10   7 Jul  19:00 IST  South Africa Women Tour Of India 2024   
11   7 Jul  20:00 IST            India Tour Of Zimbabwe 2024   
12   9 Jul  19:00 IST  South Africa Women Tour Of India 2024   
13  10 Jul  20:00 IST            India Tour Of Zimbabwe 2024   
14  13 Jul  20:00 IST            India T

In [29]:
# 3
'''
Scrape the details of State-wise GDP of India from statisticstime.com.
Url = http://statisticstimes.com/
A) Rank
B) State
C) GSDP(18-19)- at current prices
D) GSDP(19-20)- at current prices
E) Share(18-19)
F) GDP($ billion)
Note: - From statisticstimes home page you have to reach to economy page through code.
'''
def economy():
    try:
        driver=webdriver.Chrome()
        driver.get('http://statisticstimes.com')
        time.sleep(5)
        driver.maximize_window()
        economy=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div[2]/button')))
        actions = ActionChains(driver)
        actions.move_to_element(economy).perform()
        
        india= WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div[2]/div/a[3]')))
        india.click()
        time.sleep(3)
        states= WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[2]/div[2]/ul/li[1]/a')))
        states.click()
        time.sleep(2)
        
        Rank=[]
        States=[]
        GSDP_21_22=[]
        GSDP_22_23=[]
        GSDP_23_24=[]
        Share_21_22=[]
        GDP_2021=[]
        
        info=WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@id="main"]/div[5]/div/div/table/tbody/tr')))
        for detail in info:
            try:
                rank=detail.find_element(By.XPATH,'.//td[1]')
                Rank.append(rank.text)
            except Exception:
                Rank.append('--')
                
            try:
                states=detail.find_element(By.XPATH,'./td[2]')
                States.append(states.text)
            except Exception:
                States.append('--')
                
            try:
                gsdp_21=detail.find_element(By.XPATH,'./td[5]')
                GSDP_21_22.append(gsdp_21.text)
            except Exception:
                GSDP_21_22.append('--')
                
            try:
                gsdp_22=detail.find_element(By.XPATH,'./td[4]')
                GSDP_22_23.append(gsdp_22.text)
            except Exception:
                GSDP_22_23.append('--')
                
            try:
                gsdp_23=detail.find_element(By.XPATH,'./td[3]')
                GSDP_23_24.append(gsdp_23.text)
            except Exception:
                GSDP_23_24.append('--')
                
            try:
                share_21=detail.find_element(By.XPATH,'./td[6]')
                Share_21_22.append(share_21.text)
            except Exception:
                Share_21_22.append('--')
                
            try:
                gdp_21=detail.find_element(By.XPATH,'./td[7]')
                GDP_2021.append(gdp_21.text)
            except Exception:
                GDP_2021.append('--')
                
        df=pd.DataFrame({'Rank':Rank,'States':States,'GSDP(21-22) Current':GSDP_21_22,'GSDP(22-23) Current':GSDP_22_23,'GSDP(23-24) Current':GSDP_23_24,'Share(21-22)':Share_21_22,'GDP 2021($ billion)':GDP_2021})
        print(df)
    except Exception as e:
        print('An error occur',e)
        
economy()

   Rank                     States GSDP(21-22) Current GSDP(22-23) Current  \
0     1                Maharashtra           3,108,022                   -   
1     2                 Tamil Nadu           2,071,286           2,364,514   
2     3                  Karnataka           1,978,094           2,269,995   
3     4              Uttar Pradesh           1,975,595           2,258,040   
4     5                    Gujarat           1,928,683           2,230,609   
5     6                West Bengal           1,329,238           1,531,758   
6     7                  Rajasthan           1,193,489           1,365,849   
7     8             Andhra Pradesh           1,148,471           1,303,524   
8     9                  Telangana           1,124,204           1,308,034   
9    10             Madhya Pradesh           1,092,964           1,246,471   
10   11                     Kerala             934,542           1,046,188   
11   12                      Delhi             881,336          

In [34]:
# 4
def github():
    try:
        driver=webdriver.Chrome()
        driver.get('https://github.com')
        time.sleep(3)
        driver.maximize_window()
        menu=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/button')))
        actions = ActionChains(driver)
        actions.move_to_element(menu).perform()
        time.sleep(3)
        trend=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[1]/header/div/div[2]/div/nav/ul/li[3]/div/div[3]/ul/li[2]/a')))
        trend.click()
        time.sleep(5)
        
        Repository_title=[]
        Repository_description=[]
        Contributors_count=[]
        Language_used=[]
        Url=[]
        
        info=driver.find_elements(By.XPATH,'//article[@class="Box-row"]')
        for i in info:
            title=i.find_element(By.XPATH,'.//a[@class="Link"]')
            Repository_title.append(title.text)
            link=title.get_attribute('href')
            Url.append(link)
            
            try:
                descr=i.find_element(By.XPATH,'.//p[@class="col-9 color-fg-muted my-1 pr-4"]')
                Repository_description.append(descr.text)
                
            except Exception:
                Repository_description.append('--')
                
            try:
                language=i.find_element(By.XPATH,'.//span[@itemprop="programmingLanguage"]')
                Language_used.append(language.text)
                
            except Exception:
                Language_used.append('--')
            
            xpaths = [
                        '//div[@class="Layout-sidebar"]/div/div[6]/div/h2/a/span',
                        '//div[@class="Layout-sidebar"]/div/div[5]/div/h2/a/span',
                        '//div[@class="Layout-sidebar"]/div/div[4]/div/h2/a/span'
                    ]
        
        for url in Url:
            driver.get(url)
            time.sleep(3)
            
            element_found = False
            for xpath in xpaths:
                try:
                    cont = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                    Contributors_count.append(cont.text)
                    element_found = True
                    break 
                except Exception as e:
                    continue 
            if not element_found:
                    Contributors_count.append('-')
        
        for link,title,descr,lang,contr in zip(Url,Repository_title,Repository_description,Language_used,Contributors_count):
            print('URL :',link)
            print('Repository title :',title)
            print('Repository Description :',descr)
            print('Language Used :',lang)
            print('Contributors :',contr)
            print('----------------------------------------------------------------------------------')
#             try:
#                 cont=driver.find_element(By.XPATH,'//span[@class="Counter ml-1"]')
#                 cont=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="Layout-sidebar"]/div/div[6]/div/h2/a/span')))
#                 print('cont :',cont.text)
            
#             except Exception:
#                 print('----')
    except Exception as e:
        print("An error occur ",e)
        
github()

URL : https://github.com/coollabsio/coolify
Repository title : coollabsio / coolify
Repository Description : An open-source & self-hostable Heroku / Netlify / Vercel alternative.
Language Used : PHP
Contributors : 135
----------------------------------------------------------------------------------
URL : https://github.com/idootop/mi-gpt
Repository title : idootop / mi-gpt
Repository Description : 🏠 将小爱音箱接入 ChatGPT 和豆包，改造成你的专属语音助手。
Language Used : TypeScript
Contributors : 3
----------------------------------------------------------------------------------
URL : https://github.com/xenova/transformers.js
Repository title : xenova / transformers.js
Repository Description : State-of-the-art Machine Learning for the web. Run 🤗 Transformers directly in your browser, with no need for a server!
Language Used : JavaScript
Contributors : 29
----------------------------------------------------------------------------------
URL : https://github.com/codecrafters-io/build-your-own-x
Repository tit

In [11]:
# Q5
# Scrape the details of top 100 songs on billiboard.com. Url = https:/www.billboard.com/ You have to find the following details:
# A) Song name
# B) Artist name
# C) Last week rank
# D) Peak rank
# E) Weeks on board
def songs():
    driver=webdriver.Chrome()
    driver.get('https:/www.billboard.com')
    time.sleep(5)
    driver.maximize_window()
    
    chart=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/header/div/div[2]/div/div/div[2]/div[2]/div/div/nav/ul/li[1]/a')))
    chart.click()
    time.sleep(3)
    
    hot_100=WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/header/div/div[3]/div/nav/ul/li[1]/a')))
    hot_100.click()
    time.sleep(5)
    song_name=[]
    artist_name=[]
    week_rank=[]
    peak_rank=[]
    weeks_board=[]
    
    try:
        songs=WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="o-chart-results-list-row-container"]')))
        for info in songs:
            driver.execute_script("window.scrollTo(0, window.scrollY + 130)")
            name=info.find_element(By.XPATH,'.//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]/li[1]/h3')
            song_name.append(name.text)
            ar_name=info.find_element(By.XPATH,'.//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]/li[1]/span')
            artist_name.append(ar_name.text)
            
            w_rank=info.find_element(By.XPATH,'.//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]/li[4]/span')
            week_rank.append(w_rank.text)
            
            p_rank=info.find_element(By.XPATH,'.//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]/li[5]/span')
            peak_rank.append(p_rank.text)
            
            w_board=info.find_element(By.XPATH,'.//ul[@class="lrv-a-unstyle-list lrv-u-flex lrv-u-height-100p lrv-u-flex-direction-column@mobile-max"]/li[6]/span')
            weeks_board.append(w_board.text)
            time.sleep(2)
            
        df=pd.DataFrame({'Song Name':song_name,'Artist Name':artist_name,'Last Week Rank':week_rank,'Peak Rank':peak_rank,'Weeks on board':weeks_board})
        print(df)
    except Exception as e:
        print('An error occur:',e)
        
songs()

              Song Name                                        Artist Name  \
0       I Had Some Help                Post Malone Featuring Morgan Wallen   
1           Not Like Us                                     Kendrick Lamar   
2   Million Dollar Baby                                      Tommy Richman   
3    A Bar Song (Tipsy)                                          Shaboozey   
4              Espresso                                  Sabrina Carpenter   
..                  ...                                                ...   
95             Attitude  Don Toliver Featuring Charlie Wilson & Cash Co...   
96           Heat Stick    Shoreline Mafia Presents OhGeesy & Fenix Flexin   
97          The Alchemy                                       Taylor Swift   
98       On One Tonight                                              Gunna   
99              Tu Name                                      Fuerza Regida   

   Last Week Rank Peak Rank Weeks on board  
0               1 

In [17]:
# Q6
'''
Scrape the details of Highest selling novels.
A) Book name
B) Author name
C) Volumes sold
D) Publisher
E) Genre
Url - https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-grey-compare
'''
def novels():
    driver=webdriver.Chrome()
    driver.get('https://www.theguardian.com/news/datablog/2012/aug/09/best-selling-books-all-time-fifty-shades-grey-compare')
    time.sleep(3)
    
    book_name=[]
    author_name=[]
    volumes_sold=[]
    publisher=[]
    genre=[]
    try:
        info= WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, '//table[@class="in-article sortable"]/tbody/tr')))
        for detail in info:
            book= detail.find_element(By.XPATH,'.//td[2]')
            book_name.append(book.text)
            
            author=detail.find_element(By.XPATH,'.//td[3]')
            author_name.append(author.text)
            
            volumes=detail.find_element(By.XPATH,'.//td[4]')
            volumes_sold.append(volumes.text)
            
            publish=detail.find_element(By.XPATH,'.//td[5]')
            publisher.append(publish.text)
            
            gen=detail.find_element(By.XPATH,'.//td[6]')
            genre.append(gen.text)
            
            time.sleep(2)
        df=pd.DataFrame({'Book Name':book_name,'Author Name':author_name,'Volume Sold':volumes_sold,'Publisher':publisher,'Genre':genre})
        print(df)
    except Exception as e:
        print('An error occur:',e)
        
novels()

                                            Book Name       Author Name  \
0                                   Da Vinci Code,The        Brown, Dan   
1                Harry Potter and the Deathly Hallows     Rowling, J.K.   
2            Harry Potter and the Philosopher's Stone     Rowling, J.K.   
3           Harry Potter and the Order of the Phoenix     Rowling, J.K.   
4                                Fifty Shades of Grey      James, E. L.   
..                                                ...               ...   
95                                          Ghost,The    Harris, Robert   
96                     Happy Days with the Naked Chef     Oliver, Jamie   
97              Hunger Games,The:Hunger Games Trilogy  Collins, Suzanne   
98  Lost Boy,The:A Foster Child's Search for the L...      Pelzer, Dave   
99  Jamie's Ministry of Food:Anyone Can Learn to C...     Oliver, Jamie   

   Volume Sold        Publisher                        Genre  
0    5,094,805       Transworld  Cri

In [44]:
# 7
def dataset():
    try:
        driver=webdriver.Chrome()
        driver.get('https://www.imdb.com')
        driver.maximize_window()
        time.sleep(2)
        colapse= WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/nav/div[2]/label[1]"))
        )
        colapse.click()
        time.sleep(2)
        tv_show=WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/nav/div[2]/aside[1]/div/div[2]/div/div[2]/div[1]/span/div/div/ul/a[3]"))
        )
        tv_show.click()
        
        time.sleep(2)
        
        Name=[]
        Year_span=[]
        Genre=[]
        Run_time=[]
        Ratings=[]
        Votes=[]
    
        info=WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="ipc-metadata-list-summary-item sc-10233bc-0 iherUv cli-parent"]')))
        try:
            for i in info:
                time.sleep(1)
                driver.execute_script("window.scrollTo(0, window.scrollY + 110)")
                name= WebDriverWait(i, 10).until(
                    EC.visibility_of_element_located((By.XPATH, './/h3[@class="ipc-title__text"]'))
                )
                Name.append(name.text)
                year= WebDriverWait(i, 10).until(
                    EC.visibility_of_element_located((By.XPATH, './/div[@class="sc-b189961a-7 feoqjK cli-title-metadata"]/span[1]'))
                )
                Year_span.append(year.text)
                Voting= WebDriverWait(i, 10).until(
                    EC.visibility_of_element_located((By.XPATH, './/span[@class="ipc-rating-star--voteCount"]'))
                )
                Votes.append(Voting.text.strip('()')[2:])
                
                Rating= WebDriverWait(i, 10).until(
                    EC.visibility_of_element_located((By.XPATH, './/span[@class="ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"]'))
                )
                Ratings.append(Rating.text.split('(')[0])
                
                link=WebDriverWait(i, 10).until(
                    EC.element_to_be_clickable((By.XPATH, './/button[@class="ipc-icon-button cli-info-icon ipc-icon-button--base ipc-icon-button--onAccent2"]'))
                )
                driver.execute_script("arguments[0].click();", link)
                
                try:
                    genre1= WebDriverWait(i,10).until(
                        EC.visibility_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[2]/li[1]"))
                    )
                    text1=genre1.text
                except NoSuchElementException:
                    text1='-'
                    
                try:
                    genre2= WebDriverWait(i, 10).until(
                        EC.visibility_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[2]/li[2]"))
                    )
                    text2=genre2.text
                except :
                    text2='-'
                    
                try:
                    genre3= WebDriverWait(i, 10).until(
                        EC.visibility_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div/div[2]/div/div/div[1]/div[2]/ul[2]/li[3]"))
                    )                      
                    text3=genre3.text
                except Exception:
                    text3='.'
                    
                text=text1+""+text2+""+text3
                Genre.append(text)
                  
                try:
                    run_time=WebDriverWait(i, 10).until(
                                EC.visibility_of_element_located((By.XPATH, '//div[@class="sc-a78ec4e3-2 gmHPTF"]/ul[1]/li[2]'))
                            )
                    Run_time.append(run_time.text)
                except Exception:
                    Run_time.append('-')
                time.sleep(2)
                close=i.find_element(By.XPATH,'/html/body/div[4]/div[2]/div/div[1]/button')
                driver.execute_script("arguments[0].click();", close)
                time.sleep(2)
                
#             print(f"Name: {len(Name)} Year: {len(Year_span)} Genre : {len(Genre)} Run : {len(Run_time)} Rating : {len(Ratings)} Voting :{len(Votes)} ")
            df=pd.DataFrame({'Name':Name,'Year Span':Year_span,'Genre':Genre,'Run Time':Run_time,'Ratings':Ratings,'Votes':Votes})
            print(df)
        except TimeoutException as t:
            print('time error',t)
        except NoSuchElementException as f:
            print('empty list',f)
        except StaleElementReferenceException as s:
            print('stale error',s)
            
    except Exception as e:
        print('An error occur',e)
        
dataset()

               Name  Year Span                 Genre Run Time Ratings Votes
0        Bridgerton      2020–         DramaRomance.       1h  7.4\n   178K
1           Fallout      2024–  ActionAdventureDrama       1h  8.4\n   197K
2              Eric       2024    CrimeDramaThriller    53min  7.0\n    10K
3             Tires  2024–2025         ActionComedy.    20min  7.4\n   5.8K
4       Dark Matter      2024–   DramaSci-FiThriller        -  7.5\n   8.3K
..              ...        ...                   ...      ...     ...   ...
95  The White Lotus  2021–2025          ComedyDrama.    TV-MA  8.0\n   211K
96        Severance      2022–    DramaMysterySci-Fi    50min  8.7\n   206K
97     Chicago P.D.      2014–      ActionCrimeDrama    43min  8.1\n    46K
98         Insomnia      2024–            Thriller-.        -  5.4\n    212
99          M*A*S*H  1972–1983        ComedyDramaWar    25min  8.5\n    64K

[100 rows x 6 columns]


In [20]:
# 8
def ml_dataset():
    try:
        driver=webdriver.Chrome()
        driver.get('https://archive.ics.uci.edu')
        time.sleep(5)
        all_set=WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/div[1]/div[1]/main/div/div[1]/div/div/div/a[1]"))
        )
        all_set.click()
        
        time.sleep(2)
        Dataset_name=[]
        Data_type=[]
        Task=[]
        Attribute_type=[]
        No_of_instances=[]
        Attribute=[]
        Year=[]        
        time.sleep(2)
        
        for k in range(20):
            time.sleep(3)
            driver.execute_script("window.scrollTo(0, document.body.scrollTop);") 
            time.sleep(2)
            all_info=WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="rounded-box bg-base-100"]')))
            for i in all_info:
                driver.execute_script("window.scrollTo(0, window.scrollY + 100)")
                time.sleep(5)
                name=i.find_element(By.XPATH,'.//h2[@class="truncate text-primary"]/a')
                Dataset_name.append(name.text)
                print('Dataset Name :',name.text)

                data_type=i.find_element(By.XPATH,'.//div[@class="my-2 hidden gap-4 md:grid grid-cols-12"]/div[1]/span')
                Data_type.append(data_type.text)
                print('Data Type : ',data_type.text)

                attr_type=i.find_element(By.XPATH,'.//div[@class="my-2 hidden gap-4 md:grid grid-cols-12"]/div[2]/span')
                Attribute_type.append(attr_type.text)
                print('Attribute Type : ',attr_type.text)

                no_instance=i.find_element(By.XPATH,'.//div[@class="my-2 hidden gap-4 md:grid grid-cols-12"]/div[3]/span')
                No_of_instances.append(no_instance.text)
                print('No. Instances : ',no_instance.text)

                attr=i.find_element(By.XPATH,'.//div[@class="my-2 hidden gap-4 md:grid grid-cols-12"]/div[4]/span')
                Attribute.append(attr.text)
                print('Attribute : ',attr.text)

                task=i.find_element(By.XPATH,'.//div[@class="relative col-span-8 sm:col-span-7"]/p')
                Task.append(task.text)
                print('Task : ',task.text)
                colspan=i.find_element(By.XPATH,'.//div[@class="rounded-box grid grid-cols-8 gap-2 p-2 hover:bg-base-200"]')
                colspan.click()

                time.sleep(2)

                date=driver.find_element(By.XPATH,'//table[@class="col-span-full my-2 table sm:col-start-2"]/tbody/tr/td[3]')
                Year.append(date.text)
                print('Year : ',date.text)

                colspan_close=i.find_element(By.XPATH,'.//div[@class="rounded-box grid grid-cols-8 gap-2 p-2 hover:bg-base-200"]')
                colspan_close.click()
            print(f'-------------------------Page : {k+1}-------------------------')
                
            try:
                next_btn=WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, '//div[@class="btn-group"]/button[2]'))
                )
                time.sleep(2)
                next_btn.click()
            except:
                print('next button not found')
                break
        print(f"Name : {len(Dataset_name)} Type: {len(Data_type)} Attr type :{len(Attribute_type)} No. ins :{len(No_of_instances)} Attr : {len(Attribute)} Task: {len(Task)} Date : {len(Year)}")
            

    except Exception as e:
        print('An error occur :',e)
        
ml_dataset()

Dataset Name : Iris
Data Type :  Classification
Attribute Type :  Tabular
No. Instances :  150 Instances
Attribute :  4 Features
Task :  A small classic dataset from Fisher, 1936. One of the earliest known datasets used for evaluating classification methods.
Year :  7/1/1988
Dataset Name : Dry Bean
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  13.61K Instances
Attribute :  16 Features
Task :  Images of 13,611 grains of 7 different registered dry beans were taken with a high-resolution camera. A total of 16 features; 12 dimensions and 4 shape forms, were obtained from the grains.
Year :  9/14/2020
Dataset Name : Heart Disease
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  303 Instances
Attribute :  13 Features
Task :  4 databases: Cleveland, Hungary, Switzerland, and the VA Long Beach
Year :  7/1/1988
Dataset Name : Rice (Cammeo and Osmancik)
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  3.81K Insta

Year :  1/26/2016
Dataset Name : Glass Identification
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  214 Instances
Attribute :  9 Features
Task :  From USA Forensic Science Service; 6 types of glass; defined in terms of their oxide content (i.e. Na, Fe, K, etc)
Year :  9/1/1987
Dataset Name : Human Activity Recognition Using Smartphones
Data Type :  Classification, Clustering
Attribute Type :  Multivariate, Time-Series
No. Instances :  10.3K Instances
Attribute :  561 Features
Task :  Human Activity Recognition database built from the recordings of 30 subjects performing activities of daily living (ADL) while carrying a waist-mounted smartphone with embedded inertial sensors.
Year :  12/10/2012
Dataset Name : Bike Sharing
Data Type :  Regression
Attribute Type :  Multivariate
No. Instances :  17.39K Instances
Attribute :  13 Features
Task :  This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in Capital bikeshare sys

Year :  10/1/1987
Dataset Name : Hepatitis
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  155 Instances
Attribute :  19 Features
Task :  From G.Gong: CMU; Mostly Boolean or numeric-valued attribute types; Includes cost data (donated by Peter Turney)
Year :  11/1/1988
-------------------------Page : 5-------------------------
Dataset Name : Real Estate Valuation
Data Type :  Regression
Attribute Type :  Multivariate
No. Instances :  414 Instances
Attribute :  6 Features
Task :  The real estate valuation is a regression problem. The market historical data set of real estate valuation are collected from Sindian Dist., New Taipei City, Taiwan.
Year :  8/18/2018
Dataset Name : Seeds
Data Type :  Classification, Clustering
Attribute Type :  Multivariate
No. Instances :  210 Instances
Attribute :  
Task :  Measurements of geometrical properties of kernels belonging to three different varieties of wheat. A soft X-ray technique and GRAINS package were used to constr

Year :  5/31/2015
Dataset Name : Banknote Authentication
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  1.37K Instances
Attribute :  
Task :  Data were extracted from images that were taken for the evaluation of an authentication procedure for banknotes.
Year :  4/16/2013
Dataset Name : Breast Cancer Wisconsin (Prognostic)
Data Type :  Classification, Regression
Attribute Type :  Multivariate
No. Instances :  198 Instances
Attribute :  
Task :  Prognostic Wisconsin Breast Cancer Database
Year :  12/1/1995
Dataset Name : Balance Scale
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  625 Instances
Attribute :  4 Features
Task :  Balance scale weight & distance database
Year :  4/22/1994
Dataset Name : Appliances Energy Prediction
Data Type :  Regression
Attribute Type :  Multivariate, Time-Series
No. Instances :  19.74K Instances
Attribute :  29 Features
Task :  Experimental data used to create regression models of appliances energy

Dataset Name : Horse Colic
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  368 Instances
Attribute :  27 Features
Task :  Well documented attributes; 368 instances with 28 attributes (continuous, discrete, and nominal); 30% missing values
Year :  8/6/1989
Dataset Name : KDD Cup 1999 Data
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  4M Instances
Attribute :  42 Features
Task :  This is the data set used for The Third International Knowledge Discovery and Data Mining Tools Competition, which was held in conjunction with KDD-99
Year :  1/1/1999
Dataset Name : MetroPT-3 Dataset
Data Type :  Classification
Attribute Type :  Tabular, Multivariate, Time-Series
No. Instances :  1.52M Instances
Attribute :  15 Features
Task :  From a metro train in an operational context, readings from pressure, temperature, motor current, and air intake valves were collected from a compressor's Air Production Unit (APU). This dataset reveals real predi

Task :  This dataset is randomly collected from an Iranian telecom company's database over a period of 12 months.
Year :  4/9/2020
Dataset Name : ILPD (Indian Liver Patient Dataset)
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  583 Instances
Attribute :  10 Features
Task :  Death by liver cirrhosis continues to increase, given the increase in alcohol consumption rates, chronic hepatitis infections, and obesity-related liver disease. Notwithstanding the high mortality of this disease, liver diseases do not affect all sub-populations equally. The early detection of pathology is a determinant of patient outcomes, yet female patients appear to be marginalized when it comes to early diagnosis of liver pathology. The dataset comprises 584 patient records collected from the NorthEast of Andhra Pradesh, India. The prediction task is to determine whether a patient suffers from liver disease based on the information about several biochemical markers, including album

Task :  The DARWIN dataset includes handwriting data from 174 participants. The classification task consists in distinguishing Alzheimer’s disease patients from healthy people.
Year :  5/12/2022
Dataset Name : Molecular Biology (Splice-junction Gene Sequences)
Data Type :  Classification
Attribute Type :  Sequential, Domain-Theory
No. Instances :  3.19K Instances
Attribute :  61 Features
Task :  Primate splice-junction gene sequences (DNA) with associated imperfect domain theory
Year :  1/1/1992
Dataset Name : Travel Reviews
Data Type :  Classification, Clustering
Attribute Type :  Multivariate, Text
No. Instances :  980 Instances
Attribute :  11 Features
Task :  Reviews on destinations in 10 categories mentioned across East Asia. Each traveler rating is mapped as Excellent(4), Very Good(3), Average(2), Poor(1), and Terrible(0) and average rating is used.
Year :  12/19/2018
Dataset Name : Connect-4
Data Type :  Classification
Attribute Type :  Multivariate, Spatial
No. Instances :  67.

Year :  10/31/2023
Dataset Name : Challenger USA Space Shuttle O-Ring
Data Type :  Regression
Attribute Type :  Multivariate
No. Instances :  23 Instances
Attribute :  4 Features
Task :  Task: predict the number of O-rings that experience thermal distress on a flight at 31 degrees F given data on the previous 23 shuttle flights
Year :  8/5/1993
Dataset Name : Cardiotocography
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  2.13K Instances
Attribute :  
Task :  The dataset consists of measurements of fetal heart rate (FHR) and uterine contraction (UC) features on cardiotocograms classified by expert obstetricians.
Year :  9/7/2010
Dataset Name : Annealing
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  798 Instances
Attribute :  38 Features
Task :  Steel annealing data
Year :  N/A
Dataset Name : Lymphography
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  148 Instances
Attribute :  
Task :  This lymphogr

Year :  2/23/2023
Dataset Name : Cylinder Bands
Data Type :  Classification
Attribute Type :  Multivariate
No. Instances :  512 Instances
Attribute :  39 Features
Task :  Used in decision tree induction for mitigating process delays known as cylinder bands" in rotogravure printing"
Year :  8/1/1995
-------------------------Page : 19-------------------------
Dataset Name : Daily Demand Forecasting Orders
Data Type :  Regression
Attribute Type :  Time-Series
No. Instances :  60 Instances
Attribute :  13 Features
Task :  The dataset was collected during 60 days, this is a real database of a brazilian logistics company.
Year :  11/21/2017
Dataset Name : Gender by Name
Data Type :  Classification, Clustering
Attribute Type :  Text
No. Instances :  147.27K Instances
Attribute :  4 Features
Task :  This dataset attributes first names to genders, giving counts and probabilities. It combines open-source government data from the US, UK, Canada, and Australia.
Year :  3/15/2020
Dataset Name : EEG